# Mesh to Skeleton

This notebook contains code for making segmentation data into meshes into cloudvolume skeletons into erlgraphs that can be used to calculate ERL.

## Imports & Functions

In [1]:
import numpy as np 
from glob import glob
from cloudvolume import Skeleton
from zmesh import Mesher
import skeletor as sk
import trimesh
from em_erl.erl import skel_to_erlgraph


In [2]:
def load_data(path):
    """
    Helper function to load all npy and npz files in a directory into a single numpy array.
    Requires all files to have the same shape.

    Parameters
    ----------
    path : str
        The path to the directory containing the npy and npz files.

    Returns
    -------
    np.ndarray
        A numpy array containing the loaded data.
    """
    files = sorted(glob(path + "*.npy"))

    data_shape = np.load(files[0]).shape

    out = np.zeros((data_shape[0], data_shape[1], len(files)))

    for i, f in enumerate(files):
        out[:, :, i] = np.load(f)

    return out.astype(int)

## Loading data

Data can be downloaded via data_download.ipynb

In [3]:
# Load data and some preprocessing to save memory
gt_seg = load_data("data/microns/seg_v1078/")
# Data are arrays of large int64. For small volumes this is unnecessary
# Make segmentation numbers smaller
gt_seg = gt_seg - (np.min(gt_seg[np.nonzero(gt_seg)]) - 1)
# transform to uint16
gt_seg = gt_seg.astype(np.uint16)

proof_seg = load_data('data/microns/seg_v117/')
proof_seg = proof_seg - (np.min(proof_seg[np.nonzero(proof_seg)]) - 1)
proof_seg = proof_seg.astype(np.uint16)

## Making proofread skeletons

In [5]:
# Make segmentations into dict of meshes
proof_mesher = Mesher((4,4,40))

proof_mesher.mesh(proof_seg, close=True)

proof_meshes = dict()

for obj_id in proof_mesher.ids():
  mesh = proof_mesher.get(
      obj_id, 
      normals=False, # whether to calculate normals or not
      # tries to reduce triangles by this factor
      # 0 disables simplification
      reduction_factor=0, 
      # Max tolerable error in physical distance
      # note: if max_error is not set, the max error
      # will be set equivalent to one voxel along the 
      # smallest dimension.
      max_error=8,
      # whether meshes should be centered in the voxel
      # on (0,0,0) [False] or (0.5,0.5,0.5) [True]
      voxel_centered=False, 
    )
  proof_meshes[obj_id] = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)
  proof_mesher.erase(obj_id) # delete high res mesh
  del mesh

In [7]:
# Make meshes into skeletons
proof_skeletons = dict()

mesh_keys = list(proof_meshes.keys())
for id in mesh_keys:
    mesh = proof_meshes.pop(id)
    #mesh = sk.pre.fix_mesh(mesh, remove_disconnected=5, inplace=False)
    skel = sk.skeletonize.by_wavefront(mesh, waves=1, step_size=1)
    # save as cloudvolume Skeleton object
    proof_skeletons[id] = Skeleton(vertices=skel.vertices, edges=skel.edges, segid=id)
    # free up space
    del mesh
    del skel

Skeletonizing:   0%|          | 0/34 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/164 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/1166 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/438 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/3150 [00:00<?, ?it/s]

...

Skeletonizing:   0%|          | 0/39106 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/30848 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/1658 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/31408 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/42072 [00:00<?, ?it/s]

In [8]:
proof_skeletons

{37633: Skeleton(segid=37633, vertices=(shape=6, float32), edges=(shape=5, uint32), radius=(6, float32), vertex_types=(6, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 17025: Skeleton(segid=17025, vertices=(shape=13, float32), edges=(shape=12, uint32), radius=(13, float32), vertex_types=(13, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 44673: Skeleton(segid=44673, vertices=(shape=52, float32), edges=(shape=51, uint32), radius=(52, float32), vertex_types=(52, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 7937: Skeleton(segid=7937, vertices=(shape=23, float32), edges=(shape=22, uint32), radius=(23, float32), vertex_types=(23, uint8), space='voxel' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]]),
 30593: Skeleton(segid=30593, vertices=(shape=166, float32), edges=(shape=165, uint32), radius=(16

## Making ground truth skeletons

In [9]:
# Same for ground truth
gt_mesher = Mesher((4,4,40))

gt_mesher.mesh(gt_seg, close=True)

gt_meshes = dict()

for obj_id in gt_mesher.ids():
  mesh = gt_mesher.get(
      obj_id, 
      normals=False, # whether to calculate normals or not

      # tries to reduce triangles by this factor
      # 0 disables simplification
      reduction_factor=0, 

      # Max tolerable error in physical distance
      # note: if max_error is not set, the max error
      # will be set equivalent to one voxel along the 
      # smallest dimension.
      max_error=8,
      # whether meshes should be centered in the voxel
      # on (0,0,0) [False] or (0.5,0.5,0.5) [True]
      voxel_centered=False, 
    )
  gt_meshes[obj_id] = trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)
  mesher.erase(obj_id) # delete high res mesh
  del mesh

In [10]:
gt_skels = dict()

mesh_keys = list(gt_meshes.keys())
for id in mesh_keys:
    mesh = gt_meshes[id]
    #mesh = sk.pre.fix_mesh(mesh, remove_disconnected=5, inplace=False)
    skel = sk.skeletonize.by_wavefront(mesh, waves=1, step_size=1)
    gt_skels[id] = Skeleton(vertices=skel.vertices, edges=skel.edges, segid=id)
    del mesh
    del skel

Skeletonizing:   0%|          | 0/144 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/164 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/1166 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/246 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/3150 [00:00<?, ?it/s]

...

Skeletonizing:   0%|          | 0/14744 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/19906 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/73520 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/12042 [00:00<?, ?it/s]

Skeletonizing:   0%|          | 0/25416 [00:00<?, ?it/s]

In [9]:
gt_graph = skel_to_erlgraph(gt_skels)
proof_graph = skel_to_erlgraph(proof_skeletons)